In [1]:
# !pip install tensorflow

In [2]:
# !pip install tensorflow_addons

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, PReLU, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import SGD

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss

from tqdm.auto import tqdm

np.random.seed(12345)
tf.random.set_seed(12345)

/Users/krinnirk/opt/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


# ========== Data ==========

In [4]:
data = pd.read_csv('dataset3.csv')

In [5]:
data

,x1,x2,y
0,0,4,A
1,2,2,B
2,2,4,A
3,1,4,A
4,-1,3,A
5,0,2,A
6,3,1,B
7,-3,1,C
8,3,3,B
9,1,-2,C


In [6]:
X = data[['x1', 'x2']]

In [7]:
y = data['y']

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True)

In [9]:
enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
y_enc_train = enc.fit_transform(y_train.to_numpy().reshape(-1, 1))
y_enc_val = enc.transform(y_val.to_numpy().reshape(-1, 1))

# ========== Model ==========

In [10]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [11]:
devices_type = [device.device_type for device in tf.config.list_physical_devices()]

devices_type

['CPU']

In [12]:
if 'GPU' in devices_type:
    device_name = '/device:GPU:0'
else:
    device_name = '/device:CPU:0'

In [13]:
best_loss = np.inf

dropout_rates = [0, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

for dropout_rate in tqdm(dropout_rates):
    model = Sequential()

    model.add(Dense(5, input_dim=X.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
              
    model.add(Dense(4, activation='relu'))
    model.add(Dropout(dropout_rate))
    
    model.add(Dense(y_enc_train.shape[1], activation='softmax'))

    model.compile(loss=CategoricalCrossentropy(), optimizer=SGD(learning_rate=1))

    tqdm_callback = tfa.callbacks.TQDMProgressBar(show_epoch_progress=False)

    with tf.device(device_name):
        history = model.fit(X_train, y_enc_train, epochs=500, callbacks=[tqdm_callback], verbose=0)  

    y_pred_enc_val = model.predict(X_val)
    if np.isnan(y_pred_enc_val.min()):
        continue
    loss = log_loss(y_val, y_pred_enc_val, labels=np.unique(y))
    
    if loss < best_loss:
        best_loss = loss
        best_model = model
        best_dropout_rate = dropout_rate

  0%|          | 0/8 [00:00<?, ?it/s]

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

Training:   0%|           0/500 ETA: ?s,  ?epochs/s

In [14]:
best_dropout_rate

0.25

In [15]:
best_model.get_weights()

[array([[-0.05417024, -1.5710509 ,  1.7611705 , -1.216267  , -1.5252236 ],
        [-1.9788636 , -1.8414538 , -0.9809392 ,  2.155304  , -1.4555238 ]],
       dtype=float32),
 array([-0.76215065,  1.2014221 ,  0.48096785, -2.2546518 ,  1.4078176 ],
       dtype=float32),
 array([[-0.0808524 , -0.22592425,  0.05748747, -1.3485544 ],
        [ 1.7466251 , -1.1335697 , -0.9311874 , -1.4220759 ],
        [-0.10532618, -0.44238418, -0.29585487,  0.85285157],
        [-0.77257115,  0.9101406 ,  0.7988721 , -0.1710008 ],
        [ 1.5888805 , -0.78792226, -1.2119659 , -1.280787  ]],
       dtype=float32),
 array([-0.09440616, -0.5294601 , -0.42265332, -0.01969188], dtype=float32),
 array([[-0.59763217, -0.8762083 ,  1.1525611 ],
        [ 0.8226791 , -0.63894725, -0.20639168],
        [ 0.88482314, -0.5350562 ,  0.1779039 ],
        [-1.0654339 ,  0.8831549 , -0.39127272]], dtype=float32),
 array([ 0.19712913,  0.17469022, -0.3718192 ], dtype=float32)]

# ========== Prediction ==========

In [16]:
X = pd.read_csv('incoming_data3.csv')

In [17]:
X

,x1,x2
0,2,-2
1,-2,4
2,2,1
3,1,5


In [18]:
best_model.predict(X)

array([[1.3819053e-03, 4.3289526e-04, 9.9818522e-01],
       [9.9999762e-01, 1.4750130e-09, 2.4103697e-06],
       [6.8006027e-03, 9.7160292e-01, 2.1596415e-02],
       [9.9997735e-01, 5.8527771e-08, 2.2691778e-05]], dtype=float32)

In [19]:
y_pred_enc = best_model.predict(X)
y_pred = enc.inverse_transform(y_pred_enc).reshape(-1)

y_pred

array(['C', 'A', 'B', 'A'], dtype=object)